In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models as sm
from tensorflow import keras
import tensorflow as tf
sm.set_framework('tf.keras')

Segmentation Models: using `keras` framework.


In [ ]:
def FPN(patches_addr, addr_to_save):
    model = sm.FPN('vgg16', classes=1, activation='sigmoid',encoder_weights='imagenet',pyramid_dropout=0.3)

    model_name = './net2_fpn.h5'
    model.load_weights(model_name)
    print('model is loaded')
    dataset = glob.glob(patches_addr+'*')

    for i in range(len(dataset)):
        image = cv2.imread(dataset[i])
        image = np.expand_dims(image, axis=0)
        pr_mask = model.predict(image).round()
        newimg = pr_mask[0,:,:,:]

        name = dataset[i].replace(patches_addr,addr_to_save)
        cv2.imwrite(name,newimg)
    print('prediction outputs are saved!')
    print('some examples: ')


    ids = np.random.choice(np.arange(len(glob.glob(addr_to_save+'*'))), size=100)
    num = 0
    for i in ids:
        image = cv2.imread(dataset[i])
        image = np.expand_dims(image, axis=0)
        pr_mask = model.predict(image).round()
        newimg = pr_mask[0,:,:,:]
        im = np.zeros((384,384,3))
        im[:,:,0],im[:,:,1], im[:,:,2] = newimg[:,:,0],newimg[:,:,0],newimg[:,:,0]

        if np.max(im)>0:
            num+=1
            fig, axs = plt.subplots(1, 2,figsize=(15, 10))
            axs[0].imshow(cv2.imread(dataset[i]).astype(np.uint8))
            axs[1].imshow((im*100).astype(np.uint8))

        if num == 2 :
            break
